In [ ]:
#i "nuget: /home/nicholas/PointProcessDecoder/artifacts/package/debug"

Error: System.IO.FileNotFoundException: Could not load file or assembly 'FSharp.Compiler.Service, Version=43.8.200.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a'. The system cannot find the file specified.

File name: 'FSharp.Compiler.Service, Version=43.8.200.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a'
   at Microsoft.DotNet.Interactive.PackageManagement.PackageRestoreContext..ctor(Boolean forceRestore)
   at Microsoft.DotNet.Interactive.PackageManagement.KernelExtensions.<>c__DisplayClass0_0`1.<UseNugetDirective>b__0()
   at System.Lazy`1.ViaFactory(LazyThreadSafetyMode mode)
   at System.Lazy`1.ExecutionAndPublication(LazyHelper executionAndPublication, Boolean useDefaultConstructor)
   at System.Lazy`1.CreateValue()
   at Microsoft.DotNet.Interactive.PackageManagement.KernelExtensions.<>c__DisplayClass0_0`1.<UseNugetDirective>b__5(AddPackageSource command, KernelInvocationContext context)
   at Microsoft.DotNet.Interactive.Kernel.<>c__DisplayClass57_0`1.<RegisterComm

In [2]:
#r "nuget: TorchSharp"
#r "nuget: PointProcessDecoder.Core"
#r "nuget: PointProcessDecoder.Plot"
#r "nuget: PointProcessDecoder.Simulation"
#r "nuget: OxyPlot.Core, 2.1.2"
#r "nuget: SkiaSharp.NativeAssets.Linux"
#r "nuget: HarfBuzzSharp.NativeAssets.Linux"

Restore sources /home/nicholas/PointProcessDecoder/artifacts/package/debug Installed Packages HarfBuzzSharp.NativeAssets.Linux, 8.3.0.1 OxyPlot.Core, 2.1.2 PointProcessDecoder.Core, 1.0.0 PointProcessDecoder.Plot, 1.0.0 PointProcessDecoder.Simulation, 1.0.0 SkiaSharp.NativeAssets.Linux, 3.116.1 TorchSharp, 0.105.0

Loading extensions from `/home/nicholas/.nuget/packages/skiasharp/2.88.6/interactive-extensions/dotnet/SkiaSharp.DotNet.Interactive.dll`

In [3]:
using System;
using System.IO;

using static TorchSharp.torch;

using PointProcessDecoder.Core;
using PointProcessDecoder.Plot;
using PointProcessDecoder.Simulation;
using PointProcessDecoder.Core.Estimation;
using PointProcessDecoder.Core.Transitions;
using PointProcessDecoder.Core.Encoder;
using PointProcessDecoder.Core.Decoder;

In [4]:
string EncodeToHtml(byte[] data)
{
    return $"<img src=\"data:image/png;base64,{Convert.ToBase64String(data)}\">";
}

In [5]:
Tensor ReadBinaryFile(string binary_file)
{
    byte[] fileBytes = File.ReadAllBytes(binary_file);
    int elementCount = fileBytes.Length / sizeof(double);
    double[] doubleArray = new double[elementCount];
    Buffer.BlockCopy(fileBytes, 0, doubleArray, 0, fileBytes.Length);
    Tensor t = tensor(doubleArray, dtype: ScalarType.Float64);
    return t;
}

var nose_x = ReadBinaryFile("/home/nicholas/machinelearning-examples/datasets/decoder_data/nose_x.bin");
var nose_y = ReadBinaryFile("/home/nicholas/machinelearning-examples/datasets/decoder_data/nose_y.bin");
var position = stack(new Tensor[] {nose_x, nose_y}, 1);

In [6]:
ScatterPlot positionPlot = new();
positionPlot.Show(position);
EncodeToHtml(positionPlot.ToBytes());

Error: (3,27): error CS1061: 'ScatterPlot' does not contain a definition for 'ToBytes' and no accessible extension method 'ToBytes' accepting a first argument of type 'ScatterPlot' could be found (are you missing a using directive or an assembly reference?)

In [22]:
display(position.min(0).Item1);
display(position.max(0).Item1);

[2], type = Float64, device = cpu
 0 0

[2], type = Float64, device = cpu
 119.99 107.12

In [14]:
private int seed = 0;
private ScalarType scalarType = ScalarType.Float32;
private Device device = CPU;
private int testingSteps = 1800;
private string outputDirectory = "TestModel";

In [ ]:
var model = new PointProcessModel(
    EstimationMethod.KernelDensity,
    TransitionsType.Uniform,
    EncoderType.SortedSpikeEncoder,
    DecoderType.SortedSpikeDecoder,
    [0, 0],
    [120, 110],
    [50, 50],
    [1.0, 1.0],
    2,
    nUnits: 100
);